## 1. 使用分组值填充缺失值
- 对于有缺失值的数据，可以使用`fillna()`使用指定的数据去填充缺失值
- 对于分组后的各组数据里的缺失值，如果需要使用与所属分组有关的数据来填充缺失值，可以使用`apply()`调用一个填充缺失值功能的函数来处理
- 也可以使用指定的值去填充不同分组的缺失值

In [1]:
import numpy as np
import pandas as pd

In [23]:
s=pd.Series(np.random.randn(10))
s[[2,5,8,9]]=np.nan
s

0    1.094452
1   -0.076350
2         NaN
3    0.628606
4   -1.254278
5         NaN
6    0.866847
7    0.010724
8         NaN
9         NaN
dtype: float64

In [24]:
# 使用平均值填充所有缺失值
s.fillna(s.mean())

0    1.094452
1   -0.076350
2    0.211667
3    0.628606
4   -1.254278
5    0.211667
6    0.866847
7    0.010724
8    0.211667
9    0.211667
dtype: float64

In [25]:
# 对数据根据索引进行奇偶分组
f=lambda x:'偶数' if x%2==0 else '奇数'
s.groupby(f).agg(['mean','count','sum'])

,mean,count,sum
偶数,0.235674,3,0.707021
奇数,0.187660,3,0.562980


In [26]:
# fillna_s：使用传入的数据的平均值来填充数据的缺失值
def fillna_s(x):
    return x.fillna(x.mean())

# 使用apply，将groupby后的每个分组分别传给fillna_s函数
# 即使用每个分组的平均值来填充该分组的缺失值
s.groupby(f).apply(fillna_s)

0    1.094452
1   -0.076350
2    0.235674
3    0.628606
4   -1.254278
5    0.187660
6    0.866847
7    0.010724
8    0.235674
9    0.187660
dtype: float64

In [30]:
# 为每个分组指定不同的缺失值
fill_value={'奇数':99,'偶数':88}
fna=lambda x:x.fillna(fill_value[x.name])
s.groupby(f).apply(fna)

0     1.094452
1    -0.076350
2    88.000000
3     0.628606
4    -1.254278
5    99.000000
6     0.866847
7     0.010724
8    88.000000
9    99.000000
dtype: float64

In [59]:
card_type=list('桃杏梅方')

In [75]:
card_letter={1:'A',11:'J',12:'Q',13:'K'}
card_num=[card_letter[x] if x==1 or x>10 else str(x) for x in np.arange(1,14)]
card_num

['A', '2', '3', '4', '5', '6', '7', '8', '9', '10', 'J', 'Q', 'K']

In [81]:
cards=pd.DataFrame(np.array(card_num*4).reshape(4,13),index=card_type,columns=np.arange(1,14))
cards

,1,2,3,4,5,6,7,8,9,10,11,12,13
桃,A,2,3,4,5,6,7,8,9,10,J,Q,K
杏,A,2,3,4,5,6,7,8,9,10,J,Q,K
梅,A,2,3,4,5,6,7,8,9,10,J,Q,K
方,A,2,3,4,5,6,7,8,9,10,J,Q,K


In [97]:
cards=cards.apply(lambda card:card+card.index)

In [98]:
cards

,1,2,3,4,5,6,7,8,9,10,11,12,13
桃,A桃,2桃,3桃,4桃,5桃,6桃,7桃,8桃,9桃,10桃,J桃,Q桃,K桃
杏,A杏,2杏,3杏,4杏,5杏,6杏,7杏,8杏,9杏,10杏,J杏,Q杏,K杏
梅,A梅,2梅,3梅,4梅,5梅,6梅,7梅,8梅,9梅,10梅,J梅,Q梅,K梅
方,A方,2方,3方,4方,5方,6方,7方,8方,9方,10方,J方,Q方,K方


In [102]:
cards.sample(5,axis=1)

,13,7,12,5,3
桃,K桃,7桃,Q桃,5桃,3桃
杏,K杏,7杏,Q杏,5杏,3杏
梅,K梅,7梅,Q梅,5梅,3梅
方,K方,7方,Q方,5方,3方


In [108]:
cards.loc['梅']

1      A梅
2      2梅
3      3梅
4      4梅
5      5梅
6      6梅
7      7梅
8      8梅
9      9梅
10    10梅
11     J梅
12     Q梅
13     K梅
Name: 梅, dtype: object

In [109]:
for x in np.arange(10):
    num=np.random.randint(1,13)
    t=card_type[np.random.randint(4)]
    print(cards.loc[t,num])

6桃
7方
J方
6桃
4梅
4方
3杏
A杏
8梅
A梅
